In [108]:
import numpy as np
import pandas as pd

with open("input.txt", "r") as f:
    games = [x.strip() for x in f.readlines()]

df = pd.DataFrame({"line": games})

df[["game", "values"]] = df["line"].str.split(": ", n = 1, expand = True)
df = df.drop("line", axis = 1)

df["game"] = df["game"].str.replace("Game ", "", regex = False).astype("int")

df = df.assign(values = df["values"].str.split("; ")).explode("values").reset_index(drop = True)

df["round"] = df.index + 1

df = df.assign(values = df["values"].str.split(", ")).explode("values").reset_index(drop = True)

df[["num", "color"]] = df["values"].str.split(" ", n = 1, expand = True)
df = df.drop("values", axis = 1)
df["num"] = df["num"].astype("int")
df = df[["game", "round", "color", "num"]]

# part 1
conditions = [
    (df["color"] == "blue") & (df["num"] <= 14),
    (df["color"] == "green") & (df["num"] <= 13),
    (df["color"] == "red") & (df["num"] <= 12)
]

choices = [True, True, True]

df["possible"] = np.select(conditions, choices, default = False)

grouped = df.groupby("game").agg(n = ("game", "count"), possible = ("possible", "sum")).reset_index()

p1 = grouped[grouped.n == grouped.possible].game.sum()

print(f"part 1 solution: {p1}")

# part 2
spread = df.groupby(["game", "color"]).agg(max = ("num", "max")).reset_index().pivot(index = "game", columns = "color", values = "max").reset_index()

spread["prod"] = spread["blue"] * spread["green"] * spread["red"]

p2 = spread["prod"].sum()

print(f"part 2 solution: {p2}")

part 1 solution: 2685
part 2 solution: 83707
